# Cloud Inpainting on SEN3 Data
This notebook is a quick test to load, mask with clouds and then try and recover the data hidden by clouds using an inpainting DL model

In [1]:
import utils.prepare_dataset as predat
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from network.model import GenModel
from network.loss import lossL1


2024-06-20 00:59:20.200084: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-20 00:59:20.200588: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 00:59:20.204258: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-20 00:59:20.251046: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-20 00:59:21.162469: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

## Extracting the bands from the Xarrays

The goal here is to convert the data into a more tensorflow friendly data format

In [2]:
dsnp_c = np.moveaxis(np.load("Cloudy_DS.npy"),3,0)
dsnp = np.moveaxis(np.load("Ground_truth.npy"),3,0)

In [3]:
#Split into training and test set


train_ds = tf.data.Dataset.from_tensor_slices((dsnp_c[:800,:,:,:], dsnp[:800,:,:,:]))
test_ds = tf.data.Dataset.from_tensor_slices((dsnp_c[800:,:,:,:], dsnp[800:,:,:,:]))

In [4]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_ds = train_ds.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

In [5]:
model = GenModel()
coarseNet = model.coarseNet
coarseNet.compile(optimizer='adam', loss=lossL1, metrics=['mean_absolute_error'])
print(coarseNet.summary())


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gated_conv2d (GatedConv2D)      │ (None, 256, 256, 32)   │        17,664 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,664 (69.00 KB)

 Trainable params: 17,664 (69.00 KB)

 Non-trainable params: 0 (0.00 B)

None


In [6]:
inp = np.random.rand(1,256,256,3)
out = coarseNet.predict(inp)


ValueError: Exception encountered when calling GatedConv2D.call().

[1mInput 0 of layer "conv2d" is incompatible with the layer: expected axis -1 of input shape to have value 11, but received input with shape (1, 256, 256, 3)[0m

Arguments received by GatedConv2D.call():
  • input=tf.Tensor(shape=(1, 256, 256, 3), dtype=float32)

In [ ]:
#coarseNet.fit(train_ds, epochs=1, validation_data=test_ds)